# Power Predictor - 11-08-2024

Hello, world! In this project I put to use my knowledge of Linear Regression to build a model that predicts the power consumption for a given household based on a few factors(predictors).

To achieve this, I am gointg to use the dataset ['Smart meters in London'](https://www.kaggle.com/datasets/jeanmidev/smart-meters-in-london) by Jean- Michael which is available on [Kaggle.com](kaggle.com)

## Data Collection and Exploratory Data Analysis (EDA).

**Here are some of the questions I am trying to answer from this dataset:**

* _Is there a relationship between any of the factors presented in this data; weather, economic status/ social status of a household or whetehr or not a day is a holiday, with the power consumption for that particular day?_

* _How strong is the relationship between the power consumption of a household and each of those factors(if ther ehappen to be a realationship)? If we don't put a particular factor into consideration, would it affect the power consumption for a household?_

* _Can we predict the power consumption of a household based on these data. If so, is linear regression the appropriate model to use to make those predictions?_

There, are some of the questions we could probe from this data and of course a whole lot of inference can also be done by adequately analysing this dataset. 

Now, let's dive right into it!

In [1]:
#Import Pandas
import pandas as pd
import sqlite3
import seaborn as sns# for visualizations
import matplotlib.pyplot as plt


In [2]:
acorn_details = pd.read_csv("Powerdata/acorn_details.csv", encoding='utf-8', encoding_errors='replace')
#Pandas could not read some of the data in the file due to some encoding error, hence i have had to include the 'encoding'
#parameters withion the 'read_csv' method
acorn_details.head()

,MAIN CATEGORIES,CATEGORIES,REFERENCE,ACORN-A,ACORN-B,ACORN-C,ACORN-D,ACORN-E,ACORN-F,ACORN-G,ACORN-H,ACORN-I,ACORN-J,ACORN-K,ACORN-L,ACORN-M,ACORN-N,ACORN-O,ACORN-P,ACORN-Q
0,POPULATION,Age,Age 0-4,77.0,83.0,72.0,100.0,120.0,77.0,97.0,97.0,63.0,119.0,67.0,114.0,113.0,89.0,123.0,138.0,133.0
1,POPULATION,Age,Age 5-17,117.0,109.0,87.0,69.0,94.0,95.0,102.0,106.0,67.0,95.0,64.0,108.0,116.0,86.0,89.0,136.0,106.0
2,POPULATION,Age,Age 18-24,64.0,73.0,67.0,107.0,100.0,71.0,83.0,89.0,62.0,104.0,459.0,97.0,96.0,86.0,117.0,109.0,110.0
3,POPULATION,Age,Age 25-34,52.0,63.0,62.0,197.0,151.0,66.0,90.0,88.0,63.0,132.0,145.0,109.0,96.0,90.0,140.0,120.0,120.0
4,POPULATION,Age,Age 35-49,102.0,105.0,91.0,124.0,118.0,93.0,102.0,103.0,76.0,111.0,67.0,99.0,98.0,90.0,102.0,103.0,100.0


This file contains a summary describing the people who stay in different acorns. It paints a picture as to what each acorn looks like and the kind of people who dwell there. This is important for determining electricity consumption. I will have to figure out how to leverage these data later on. Forn now, we explore another file from our data set.

In [3]:
infor_households = pd.read_csv("Powerdata/informations_households.csv")
infor_households.head()

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,ToU,ACORN-,ACORN-,block_0
1,MAC001074,ToU,ACORN-,ACORN-,block_0
2,MAC000002,Std,ACORN-A,Affluent,block_0
3,MAC003613,Std,ACORN-A,Affluent,block_0
4,MAC003597,Std,ACORN-A,Affluent,block_0


So, I am also simultaneously trying to understand the structure of the UK society with regrads to the information contained in this data set. I have been reading online as well as the notes from the Author on Kaggle as well as the discussions surrounding this dataset and I must say that, that is making the dataset easier to understand.
The file above basicall lists all the houses involved, each with a unique id(LCid), the type of tarriffs that apply to them as well as the acorn group the households are in and the file within the dataset where the household meter data is found.

In [4]:
daily_energy = pd.read_csv("Powerdata/daily_dataset/daily_dataset/block_0.csv")
daily_energy.head()

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000002,2012-10-12,0.1385,0.154304,0.886,46,0.196034,7.098,0.000
1,MAC000002,2012-10-13,0.1800,0.230979,0.933,48,0.192329,11.087,0.076
2,MAC000002,2012-10-14,0.1580,0.275479,1.085,48,0.274647,13.223,0.070
3,MAC000002,2012-10-15,0.1310,0.213688,1.164,48,0.224483,10.257,0.070
4,MAC000002,2012-10-16,0.1450,0.203521,0.991,48,0.184115,9.769,0.087


This file and the 'half_hourly' one, provide the meter readings for each household on a daily and half-hourly basis respectively. Overall, gifferent households are found in different blocks and different blocks are found in different acorn groups (which are based on the different social and economic statutes of the households).

For my application, I think I would like to make use of daily records, instaed of hourly records and the 'maximum energy consumption for a household' to be precise. This approach ensures that I include all the meter data for the day. Hourly reading can be useful, I mean they would provide a much more broadly useful model e.g. to predict energy consumption hourly or even half hourly. I might consider that as  a future project but for now, daily records seem much more meaniful for me. This also means less predictors/variables are used resulting in a less sophisticated model.

In that regard, I will use the 'daily datasets' as starting point for my analysis. The datasets, are quite too  many and huge. That could a good sign; the more the data, the better the model. But also, it is going to be a lot of data crunching, let's find out:

In [5]:
df = pd.read_csv("Powerdata/daily_dataset/daily_dataset/block_0.csv")
df

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000002,2012-10-12,0.1385,0.154304,0.886,46,0.196034,7.098,0.000
1,MAC000002,2012-10-13,0.1800,0.230979,0.933,48,0.192329,11.087,0.076
2,MAC000002,2012-10-14,0.1580,0.275479,1.085,48,0.274647,13.223,0.070
3,MAC000002,2012-10-15,0.1310,0.213688,1.164,48,0.224483,10.257,0.070
4,MAC000002,2012-10-16,0.1450,0.203521,0.991,48,0.184115,9.769,0.087
...,...,...,...,...,...,...,...,...,...
25569,MAC005492,2014-02-24,0.1690,0.175042,0.378,48,0.073174,8.402,0.079
25570,MAC005492,2014-02-25,0.1550,0.160792,0.545,48,0.082118,7.718,0.079
25571,MAC005492,2014-02-26,0.1490,0.178542,0.687,48,0.120820,8.570,0.079
25572,MAC005492,2014-02-27,0.1140,0.146167,0.478,48,0.082616,7.016,0.079


In [6]:
#Only going to make use of the maximum _daily-energy consumption per household 'energy_max'
energy_df = df.drop(['energy_count','energy_median','energy_mean','energy_std','energy_max','energy_min'],axis=1)
energy_df

,LCLid,day,energy_sum
0,MAC000002,2012-10-12,7.098
1,MAC000002,2012-10-13,11.087
2,MAC000002,2012-10-14,13.223
3,MAC000002,2012-10-15,10.257
4,MAC000002,2012-10-16,9.769
...,...,...,...
25569,MAC005492,2014-02-24,8.402
25570,MAC005492,2014-02-25,7.718
25571,MAC005492,2014-02-26,8.570
25572,MAC005492,2014-02-27,7.016


Now I have only data, important for my project. But this is only one block out of 111 blocks of households!😓

In [7]:
# To add all the other 110 blocks:
for i in range(1,112):
    temp_df = pd.read_csv(f"Powerdata/daily_dataset/daily_dataset/block_{i}.csv")
    #Delete all the other columns only keeep the useful ones:
    temp_df = temp_df.drop(['energy_count','energy_median','energy_mean','energy_std','energy_max','energy_min'],axis=1)
    #Then stack/merge the data to the energy dataframe definded above:
    energy_df = pd.concat([energy_df,temp_df],axis=0)
energy_df

,LCLid,day,energy_sum
0,MAC000002,2012-10-12,7.098
1,MAC000002,2012-10-13,11.087
2,MAC000002,2012-10-14,13.223
3,MAC000002,2012-10-15,10.257
4,MAC000002,2012-10-16,9.769
...,...,...,...
9903,MAC004828,2014-02-24,2.587
9904,MAC004828,2014-02-25,0.720
9905,MAC004828,2014-02-26,1.051
9906,MAC004828,2014-02-27,0.736


In [8]:

energy_df = energy_df.rename(columns={'LCLid':'Household_id'})#making the household id column more meaningful
#And setting the household id as the index.
energy_df = energy_df.astype({
    'Household_id': 'str',
    'day': 'object',
    'energy_sum': 'float64',
}, errors='raise')
energy_df = energy_df.set_index('Household_id')



In [9]:
try:
    energy_df.to_sql('energy',sqlite3.Connection('Powerdata.db'))
    print('dataframe converted succesffully!')
    
except ValueError as e:
    print(e)

Table 'energy' already exists.


In [10]:
df_waether = pd.read_csv("Powerdata/weather_daily_darksky.csv")
df_waether['time'] = df_waether['time'].apply(lambda x: x.split(' ')[0])
df_waether = df_waether.rename(columns={'time':'Date'})
#Now I can join this merge this data onto the joint table based on the data column
try:
    df_waether.to_sql('weather',sqlite3.Connection('Powerdata.db'))
    print('file converted succesffully!')
except ValueError as e:
    print(e)

Table 'weather' already exists.


The following sqlite querry will merge the tables as desired 

In [11]:
query = '''SELECT Household_id,day AS Date,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,precipType,visibility,humidity,uvIndex,temperatureMin,moonPhase,summary,"Type" AS Holidays,energy_sum,Acorn,Acorn_grouped,stdorToU FROM energy 
            LEFT JOIN holidays ON "Bank holidays" = energy.day 
            LEFT JOIN household_infor ON LCLid = Household_id
            LEFT JOIN weather ON weather.Date = energy.day
            '''
            

In [12]:
try:
        #Read the already saved cvs file without having to connect to the databse
        final_df = pd.read_csv('FinalPowerData.csv')
        
        
except FileNotFoundError as e:
        print(e)
        try:
                
                print('Connecting to database...')
                con = sqlite3.connect("Powerdata.db")#connect to a databse, create if not exists
                print('Database connection successful...')
                cur = con.cursor()#cursor object for interracting with the created/connected data
                print('Querrying databse...')
                final_df = pd.read_sql_query(query, con).drop_duplicates()# I just noticed that all my rows are duplictated
                print('Success! Data read from the database')
                # Close the connection
                con.close()
                print('Database disconnected...')
        except sqlite3.Error as e:
                print(f'Failure...{e}')
        
        else:
                print('Backing up to csv...')
                final_df.to_csv('FinalPowerData.csv')#Save csv back up for data persitance and time saving
                print('CSV back up complete...')
                final_df = pd.read_csv('FinalPowerData.csv')
                print('CSV back up file retrieved...')
                
else:
        print('CSV back up file retrieved')
            

[Errno 2] No such file or directory: 'FinalPowerData.csv'
Connecting to database...
Database connection successful...
Querrying databse...
Success! Data read from the database
Database disconnected...
Backing up to csv...
CSV back up complete...
CSV back up file retrieved...


In [13]:
try:
    del final_df['Unnamed: 0']
except:
    pass


final_df.set_index('Household_id')

,Date,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,precipType,visibility,humidity,uvIndex,temperatureMin,moonPhase,summary,Holidays,energy_sum,Acorn,Acorn_grouped,stdorToU
Household_id,,,,,,,,,,,,,,,,,,,
MAC000002,2012-10-12,11.53,252.0,6.15,0.29,2.18,1004.92,rain,12.94,0.84,2.0,6.41,0.92,Partly cloudy until evening.,NaN,7.098,ACORN-A,Affluent,Std
MAC000002,2012-10-13,12.32,300.0,4.10,0.20,2.36,1002.50,rain,11.86,0.80,2.0,2.81,0.96,"Partly cloudy starting in the afternoon, conti...",NaN,11.087,ACORN-A,Affluent,Std
MAC000002,2012-10-14,13.03,220.0,6.39,0.31,2.97,1002.67,rain,12.17,0.84,1.0,4.09,0.99,Partly cloudy throughout the day.,NaN,13.223,ACORN-A,Affluent,Std
MAC000002,2012-10-15,14.74,226.0,6.91,0.39,5.37,1000.80,rain,12.59,0.74,1.0,9.06,0.03,Partly cloudy throughout the day and breezy un...,NaN,10.257,ACORN-A,Affluent,Std
MAC000002,2012-10-16,16.60,178.0,9.87,0.42,4.66,998.22,rain,12.20,0.82,1.0,8.80,0.07,Partly cloudy throughout the day.,NaN,9.769,ACORN-A,Affluent,Std
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAC004828,2014-02-24,14.23,183.0,6.03,0.50,5.02,1005.19,rain,13.00,0.74,1.0,7.99,0.82,Mostly cloudy until evening and breezy overnight.,NaN,2.587,ACORN-U,ACORN-U,Std
MAC004828,2014-02-25,11.43,202.0,5.06,0.62,5.69,1000.65,rain,12.09,0.78,1.0,6.79,0.86,Mostly cloudy throughout the day.,NaN,0.720,ACORN-U,ACORN-U,Std
MAC004828,2014-02-26,11.29,227.0,2.74,0.26,3.82,1012.73,rain,13.00,0.73,2.0,4.17,0.90,Partly cloudy throughout the day.,NaN,1.051,ACORN-U,ACORN-U,Std


In [14]:
final_df.to_csv('For_Cleaning.csv')

In [ ]:
import plotly.express as px

In [ ]:
fig,ax = plt.subplots(ncols=5,figsize=(40,10))
sns.histplot(final_df['energy_sum'],kde=True,ax= ax[0])
sns.histplot(final_df['temperatureMax'],kde=True,ax=ax[1])
sns.histplot(final_df['uvIndex'],kde=True,ax=ax[2])
sns.histplot(final_df['dewPoint'],kde=True,ax=ax[3])
sns.histplot(final_df['windSpeed'],kde=True,ax=ax[4])
#px.histogram(final_df['energy_sum'])



In [ ]:
sns.boxplot(final_df['energy_sum'],orient='h')

In [ ]:
px.box(final_df['energy_sum'])

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='Date',y='energy_sum',data=final_df.rolling(30))

In [ ]:
px.line(x=final_df['Date'],y=final_df['energy_sum'])

In [ ]:
final_df['Holidays'] = final_df['Holidays'].fillna('Not a Holiday')
final_df.head()

Handling missng values

In [ ]:
final_df['temperatureMax'] = final_df['temperatureMax'].fillna(final_df['temperatureMax'].mean())
final_df.isna().sum()

In [ ]:
plt.figure(figsize=(10,6))
#plt.subplots(ncols=4)
sns.boxplot(x='Acorn_grouped', y='energy_sum',hue='Acorn_grouped',data=final_df,palette='rainbow')

In [ ]:
numerical_cols = ['windBearing','dewPoint','cloudCover','windSpeed','pressure','visibility','humidity','uvIndex','temperatureMin','moonPhase','energy_sum']
for col in numerical_cols:
    final_df[col] = final_df[col].fillna(final_df[col].mean())
    
final_df.isna().sum()

In [ ]:
final_df['summary'] = final_df['summary'].fillna('Unknown')
final_df.isna().sum()

In [ ]:
final_df['precipType'] = final_df['precipType'].fillna('None')#Just assume it never precipitated on those days
final_df.isna().sum()

Quantifying categorical data

In [ ]:
# Convert precipType to categorical
final_df['precipType'] = pd.Categorical(final_df['precipType'], categories=['None', 'rain'], ordered=True)

# Convert the categories to numeric codes
final_df['preciptation'] = final_df['precipType'].cat.codes  # Adding 1 to start the codes from 1 instead of 0

In [ ]:
final_df['Acorn_grouped'].unique()# To display the unique categories in the entire column

In [ ]:
# Convert Acorn_grouped to categorical
final_df['Acorn_grouped'] = pd.Categorical(final_df['Acorn_grouped'], categories=['ACORN-U', 'Adversity','Comfortable','Affluent','ACORN-'], ordered=True)

# Convert the categories to numeric codes
final_df['Acorns'] = final_df['Acorn_grouped'].cat.codes + 1 # Adding 1 to start the codes from 1 instead of 0


In [ ]:
# Convert 'Category' column to categorical type
final_df['Holidays'] = pd.Categorical(final_df['Holidays'])

# Define the weights for each category
weights = {cat: 1 for cat in final_df['Holidays'].cat.categories}  # Assign weight 1 to all
weights['Not a Holiday'] = 0  # Set weight of 'C' to 0

# Map the categories to their corresponding weights
final_df['Holidays'] = final_df['Holidays'].map(weights)

In [ ]:
# Convert stdorToU to categorical
final_df['stdorToU'] = pd.Categorical(final_df['stdorToU'], categories=['ToU','Std'], ordered=True)

# Convert the categories to numeric codes
final_df['Tarriff'] = final_df['stdorToU'].cat.codes + 1 # Adding 1 to start the codes from 1 instead of 0


In [ ]:
final_df.columns#Checking if I left out any column unchanged.

Now I need to create a copy of my dataframe, one that only contains the numerical values.

In [ ]:
num_df = final_df[['temperatureMax','windBearing','dewPoint','cloudCover', 'windSpeed', 'pressure' ,'visibility' ,'humidity' ,'uvIndex' ,'temperatureMin', 'moonPhase' ,'preciptation' ,'Acorns' ,'Holidays' ,'Tarriff' ,'energy_sum']]
num_df

In [ ]:

corr = num_df.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr, cmap = 'Wistia', annot= True)

In [ ]:
energy_corr = corr['energy_sum']
energy_corr

'Holidays' seems to be the least correalted with 'energy_sum' but I will keep it anyway.

In [ ]:
num_df['energy_sum'].describe()

In [ ]:
plt.figure(figsize=(10,6))
#plt.subplots(ncols=4)
sns.boxplot(x='Acorns', y='energy_sum',hue='Acorns',data=num_df,palette='rainbow')

There seem to be quite a lot of outliers especially in categories 1-4. However, I find these outliers to be reasonable values by observing my initial dataset. A small population(relatively) of rich people do have a huge daily sum of energy consumption. This is most probably due to having a lot of things and activities that consumes electricity but possibly not having technologies installed to minimize power consumption. This also is further explained by the fixed 'std' electricity tarriff that I find being paid by all 'affluent' households with energy sums above 200kwh. but for the sake of trying to build a reliable model, I will not include these households.(I have modified my sql query above to fix this). 

In [ ]:
plt.figure(figsize=(10,6))
#plt.subplots(ncols=4)
sns.boxplot(x='Tarriff', y='energy_sum',hue='Acorns',data=num_df,palette='rainbow')

In [ ]:
num_df.info()

In [ ]:
num_df.describe()

From tyhe correalation matrix /heatmap above, colinearity can be observed between uv index/dewpoint and maximum/minimum temperature. I will omit dewpoint and uv index for that matter.

In [ ]:
try:
   ...
   #num_df = num_df.drop(columns=['dewPoint','uvIndex','Holidays','windBearing','pressure','visibility','moonPhase','Tarriff','windSpeed'])
except:
   pass


In [ ]:

corr = num_df.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr, cmap = 'Wistia', annot= True)

## Modelling


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
model = LinearRegression()


In [ ]:
x = num_df.drop(columns='energy_sum')
y = num_df['energy_sum']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=23)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
prediction_df= pd.DataFrame(y_test)
prediction_df['Prediction'] = y_pred
prediction_df

In [ ]:
model.intercept_

In [ ]:
model.coef_

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
sns.scatterplot(x=y_test,y=y_pred,hue=y_test)